# HD BITCOIN WALLET

This project aims to implement and Hierarchical Deterministic(HD) wallet using the bitcoin standards BIP-32 
and BIP-39 by following the guidelines from the book Mastering Bitcoin. An HD wallet allows to derive many keys 
from a single seed stored in a tree structure. With respect to non deterministic wallets, where keys are generated 
at random and have no connection with one another, the HD wallet allows for simpler managment and doesn't require
to keep copies of all the keys since they can all be derived from the seed. Furthermore HD wallets allow to derive 
public keys without the need to access to the associated private keys which guarantees additional security in 
insecure scenarios.

![pic](CryptoProject/pic.png)

In [1]:
"""Now we import all the required libraries"""

from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac



In [2]:
"""here i create a class to store data in a tree like structure, every node has a parent, a private key,
a chain code and a list of children"""

class TreeNode():
    
    def __init__(self,parent,pvt_key,pub_key,chain_code): #every node has a parent, the master will have None as input
        self.parent=parent
        self.pvt_key=pvt_key 
        self.pub_key=pub_key
        self.chain_code =chain_code
        self.children = []
    

Now we need to follow the seed generation procedure as described by BIP-39. The whole process is performed in a few commands thanks to the Mnemonic library.The concept is of generating a random entropy which is then mapped to some words (mnemonic words), which can be used to generate the actual seed.The idea of using mnemonic words comes from the fact that they are much simpler to remember than the seed itself and can be used to unambigously retrive all of the keys in the wallet, allowing for a backup in case of loss or destruction. The whole procedure is explained in the following images
![pic](CryptoProject/pic2.png)![pic](CryptoProject/pic3.png)

In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght): #strenght refers to the lenght in bits of the entropy to generate 
                                                 #passphrase is an additional value used to append to the salt for PBKDF2
    
   mnemo = Mnemonic(language)

   words = mnemo.generate(strength=strenght) #this command performs steps 1 to 5 and saves the words in a vavariable
    
   seed = mnemo.to_seed(words, passphrase=passphrase)#here we retrieve the seed from the words (step 7 to 9)

   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
    
   print("Words= " + words)   

   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
    
   return seed 
   


In [4]:
#Here we simply test our function and check the values generated 
seed=generate_seed("english","test",128)

Entropy=b'd88fc65726462cab06abe1b8e44cb559'
Words= sudden lawsuit noise erode glare few boy label rib carbon coil rebuild
Seed = b'cdbd05675b0d60626c418bb56e8d8a9c972428973dc86dbc921f5f8ff1304e1f52853901e3d0012a8826ae019b5b41faf121c52c73b70a1b269db27f0f84be62'


Now that we have the seed it's time to generate the master. We make use of HMAC-SHA512 in order to have a fixed lenght output and to protect the seed from some possible attacks(to this end the hash is one way).It is worth noticing that the public key is then generated starting from the private key using ellipric curve theory. As the theory suggests, the public key is a pair of coordinates x and y, so to obtain the actual public key I take the first two bytes of the y coordinate and combine them with all the ones from the x axis. The following picture explains the process:
![pic](CryptoProject/pic4.png)

In [5]:
"""In this section I derive from the seed the master private key and the master chain code, 
the private key is the one used to have control over your bank account represented by the public key. 
The bitcoin address is generated/corresponds from/to the public key"""

def get_master_info(seed):
    
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest() #here we perform HMAC-SHA512 on the root seed
    
    master_pvt_key=master_hmac[:64] #we pick the first 256 bits (256/4=64 since we are working in hex) 
    
    master_chain_code=master_hmac[64:] #we extract the chain code using a similar logic 
    
    #Now we need to derive the public key for our master and I have used fastecdsa for a quick result
    
    pvt=int(master_pvt_key, 16) #some conversion for compliance with the function 
    
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1) # we generate the public key using the secp256k1 elliptic curve 
    
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #for the serialization of the public key i used the hints at: https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    
    master=TreeNode(None,master_pvt_key,pub_key,master_chain_code) #I create a TreeNode instance and pass as parent None, since we are creating the root 
    
    return master


In [6]:
master_info=get_master_info(seed)
master_info.pvt_key


'e8d5d77643db87d5614e16effaf7dcbc9f73a435b1a4cb197261633438948e7d'

Now that we have the master we have to create the children.In order to do so we need the parent's public or private key, the child index and the parent chain code. The last one is needed in order to introduce deterministic random data so that something more is required than the index and the child key to get to the siblings of a determined node. 
We have two types of children:
1) regular children which are derived using the public address of the parent
![pic](CryptoProject/pic6.png)2) hardened children which are derived from the private address of the parent
![pic](CryptoProject/pic5.png)Hardened children are particularly useful when we deal with public extended keys where we need to consider the possibility of having a children private key leaked. Since from the extended public address we can derive the chain code it is possible using the two to get the private keys of other children or even worse the one of the master. So we want to break the relationship between parent public key and child chain code.In this way the public key doesn't give any useful info on the other children private keys.In this way we can create extended public keys which are not vulnerable.


In [7]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex, num represents the number od children we want to create 
  
    for i in range(0,num): 
        
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:]) #here I concatenate all the values that should be hashed 
        
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest() #Here I perform HMAC-SHA512 on the input string
        
        pvt_key=child_info[:64] #similarly as the master we extract the private key
        
        chain_code=child_info[64:]
        
        pvt=int(pvt_key, 16) #compliance transformations 
        
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #same as master 
        
        child=TreeNode(parent.pvt_key,pvt_key,pub_key,chain_code) #I create the TreeNode entity
        
        parent.children.append(child) #I append the new entity to the list of children of the parent 

In [8]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:]) #here I concatenate all the values that should be hashed 
        
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest() #Here I perform HMAC-SHA512 on the input string
        
        pvt_key=child_info[:64] #similarly as the master we extract the private key
        
        chain_code=child_info[64:]
        
        pvt=int(pvt_key, 16) #compliance transformations 
        
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #same as master 
        
        child=TreeNode(parent.pvt_key,pvt_key,pub_key,chain_code) #I create the TreeNode entity
        
        parent.children.append(child) #I append the new entity to the list of children of the parent 
            

Now we are ready to create some children, as an example I create two regular children and and one hardened children. 

In [9]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")

Private Key acc2c444927b3968334ad726f3732a2b6f2beb59c5008ce0f52871586137a6dc
Public Key e8438513512712d42bce4ccf0853162afc52b88c2347bf21f49ddb9943dca9d301
Chain Code 4f48e7f7e8960903b2e54bbea5e75536c0bbdf251e9ee8b89c490ecb2d55807b
-------------------------------------------------------------------------
Private Key f8b78cec5a78ce9f192eb78a34712d82ed6eda602bdbb473b1c30d0cb3f92ae9
Public Key 71171132a0ee77c8abadc8d26f7f06c2bf539c015ce201608db940f298f4efb443
Chain Code ddc1f64d834fe9e7f69bae9487db8b3de2caf5f302546c93f2985ab00dfee688
-------------------------------------------------------------------------
Private Key e61bafbbce8bbe120d3d00056b5cb86f230305212665a68c51ddfba2bce07d08
Public Key 56bafc0366e1513759ae8385617b5954affa6aa57fbe3a4b6d20cdd52e52113a0a
Chain Code e9b959b1caeebb74530777e1bab37f957874d2d6a86da71aeec7ac9a40982b95
-------------------------------------------------------------------------


Now that we have some children let's create some grandkids. Each child will have one hardened children and two regular children. The result will show a total of nine nodes (3x3=9)

In [10]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 94224b109f7f33e8e6ec855f907dbc8313c4c268e4f41b7f037386703d6cee37
Public Key 2d62e74970d05188bad975615ba377278ff16948316c648cb5e4d5c2c11cb4d242
Chain Code 4cc4097010ece315e6330ef5e2065aad47b16187b2d9aa1b424d6f7719895eca
-------------------------------------------------------------------------
Private Key 29850042be1b5c2603073b9146c98e1b8257e8344bef6c9e9aa17dad79f61698
Public Key 135563bba96afea77942301d10063bde4d0a27c4a06b46e000d321e06e753ea391
Chain Code a5e3e1a384ccf387c7b970600c2ef3e3291f9934c23af71bb0d2a02921c4e1e0
-------------------------------------------------------------------------
Private Key 15007fa4c9c57eeae224edce4933f88edddb96ad6be5dcb09100884db7bfb039
Public Key 9086d8099a34ceb4833e5a4003269fc65e0acede06db99dfa3bb6fe8505480a4e2
Chain Code d42f54181a4bfa439bb885814144f8ffff54746d51a851868f8d7a79d5d0d8c5
-------------------------------------------------------------------------
Private Key c1cf3852f6d48a2d7d1235bb2c489c83b2220133c72d545ad152a42abff14c39
Public K